In [1]:
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import numpy as np
import pandas as pd
import scipy.optimize

from datetime import datetime
import pytz

import gm2
import trfp

Welcome to JupyROOT 6.12/04


In [2]:
# Blind trolley and fixed probe separately, but blind all fixed probe stations the same
# Blind dipole by order 1 ppm = 61.79 Hz
# Blind higher moments by order 100 ppb = 6.179 Hz
tr_blinds = np.zeros(17)
tr_blinds[0] = np.random.uniform(-61.79, 61.79)
tr_blinds[1:] = np.random.uniform(-6.179, 6.179, 16)
fp_blinds = np.zeros(6)
fp_blinds[0] = np.random.uniform(-61.79, 61.79)
fp_blinds[1:] = np.random.uniform(-6.179, 6.179, 5)

def blind_tr_moment_df(moment_df, fp_blinds, tr_blinds):
    out_df = moment_df.copy()
    out_df['tr,m1'] = moment_df['tr,m1'] + tr_blinds[0]
    for st in np.arange(72):
        stm = 'st' + str(st) + ',m1'
        out_df[stm] = moment_df[stm] + fp_blinds[0]
    for m in np.arange(2,7):
        trm = 'tr,m' + str(m)
        out_df[trm] = moment_df[trm] + tr_blinds[m-1]
        for st in np.arange(72):
            stm = 'st' + str(st) + ',m' + str(m)
            out_df[stm] = moment_df[stm] + fp_blinds[m-1]

    # Blind higher order trolley moments by order 100 ppb = 6.179 Hz
    for m in np.arange(7,18):
        trm = 'tr,m' + str(m)
        out_df[trm] = moment_df[trm] + tr_blinds[m-1]
        
    return out_df

def blind_fp_moment_df(moment_df, fp_blinds):
    # Blind trolley and fixed probe separately, but blind all fixed probe stations the same
    # Blind dipole by order 1 ppm = 61.79 Hz
    out_df = moment_df.copy()
    for st in np.arange(72):
        stm = 'st' + str(st) + ',m1'
        out_df[stm] = moment_df[stm] + fp_blinds[0]
    
    # Blind quads, sexts, m6 by order 100 ppb = 6.179 Hz
    for m in np.arange(2,7):
        for st in np.arange(72):
            stm = 'st' + str(st) + ',m' + str(m)
            out_df[stm] = moment_df[stm] + fp_blinds[m-1]
        
    return out_df

station_phi = trfp.STATION_BARCODE_PHI
station_edges = trfp.STATION_BARCODE_EDGES

corrected_df_1 = pd.read_hdf('60hr_trolley_runs_1.h5', key='run_3956_corrected_df')
corrected_df_1 = blind_tr_moment_df(corrected_df_1, fp_blinds, tr_blinds)
tr_baseline_1, fp_baseline_1, baseline_time_1, summed_azimuth_1, _ = trfp.trolley_run_station_average(corrected_df_1)

corrected_df_2 = pd.read_hdf('60hr_trolley_runs_2.h5', key='run_3997_corrected_df')
corrected_df_2 = blind_tr_moment_df(corrected_df_2, fp_blinds, tr_blinds)
tr_baseline_2, fp_baseline_2, baseline_time_2, summed_azimuth_2, _ = trfp.trolley_run_station_average(corrected_df_2)

# load all fixed probe runs moment_df into one big moment_df (runs 3959--3994)
print 'Appending fixed probe runs.'
fp_moment_df = pd.read_hdf('60hr_fixed_probe_runs.h5', key='run_3959_moment_df')
pts = fp_moment_df.shape[0]
for run in np.arange(3959, 3995):
    temp_df = pd.read_hdf('60hr_fixed_probe_runs.h5', key='run_'+str(run)+'_moment_df')
    pts = pts + temp_df.shape[0]
    print '\rAppending run ' + str(run) + '.',
    fp_moment_df = fp_moment_df.append(temp_df)
    
print '\nBlinding fixed probe runs.'
fp_moment_df = blind_fp_moment_df(fp_moment_df, fp_blinds)

# load all the trolley runs corrected_df into on big file (to show window of 60hr set) ()
print '\nAppending trolley runs.'
tr_corrected_df = pd.read_hdf('60hr_trolley_runs_1.h5', key='run_3955_corrected_df')
for run in np.arange(3956, 3959):
    temp_df = pd.read_hdf('60hr_trolley_runs_1.h5', key='run_'+str(run)+'_corrected_df')
    pts = pts + temp_df.shape[0]
    print '\rAppending run ' + str(run) + '.',
    tr_corrected_df = tr_corrected_df.append(temp_df)
for run in np.arange(3995, 3999):
    temp_df = pd.read_hdf('60hr_trolley_runs_2.h5', key='run_'+str(run)+'_corrected_df')
    pts = pts + temp_df.shape[0]
    print '\rAppending run ' + str(run) + '.',
    tr_corrected_df = tr_corrected_df.append(temp_df)

print '\nBlinding trolley runs.'
tr_corrected_df = blind_tr_moment_df(tr_corrected_df, fp_blinds, tr_blinds)

print '\nDone appending runs.'

### Apply baseline corrections, generate virtual trolley measurements

fp_moment_baseline = fp_moment_df.copy()
fp_moment_tr_run_baseline = tr_corrected_df.copy()

# apply baseline corrections to each fp stm
print "\nSubtracting fixed probe baselines."
for st in np.arange(72):
    num_probes = len(trfp.STATION_PROBE_ID[st])
    for m in np.arange(num_probes):
        stm = 'st'+str(st)+',m'+str(m+1)
        
        def backwards_correction(time):
            c1 = fp_baseline_1[st, m]
            c2 = fp_baseline_2[st, m]
            t1 = baseline_time_1[st]
            t2 = baseline_time_2[st]
            return (c2-c1)/(t2-t1)*(time-t1) + c1
        
        correction = backwards_correction(fp_moment_baseline.index.values)
        fp_moment_baseline[stm] = fp_moment_baseline[stm] - correction
        correction = backwards_correction(fp_moment_tr_run_baseline.index.values)
        fp_moment_tr_run_baseline[stm] = fp_moment_tr_run_baseline[stm] - correction
        
        print '\rstm: ' + stm + '.',
        
# replace columns in vtr with Jacobian-fixed columns from fp baseline correction

print "\n\nApplying Jacobian."
vtr_df = fp_moment_df.copy()
vtr_tr_run_df = tr_corrected_df.copy()

for st in np.arange(72):
    num_probes = len(trfp.STATION_PROBE_ID[st])
    if num_probes == 4:
        num_moments = 4
        if st == 41:
            J = trfp.J_4_PROBE_ST41
        elif st == 37 | st == 39:
            J = trfp.J_4_PROBE_ST37_ST39
        else:
            J = trfp.J_4_PROBE
    else:
        num_moments = 5
        if st < 7:
            J = trfp.J_6_PROBE_OFFSET
        else:
            J = trfp.J_6_PROBE
    # run over each vtr moment:
    for m in np.arange(num_moments):
        vtr_stm = 'st'+str(st)+',m'+str(m+1)
        fp_stm = ['st'+str(st)+',m'+str(fp_m+1) for fp_m in np.arange(num_moments)]
        vtr_df[vtr_stm] = fp_moment_baseline[fp_stm].dot(J[m])
        vtr_tr_run_df[vtr_stm] = fp_moment_tr_run_baseline[fp_stm].dot(J[m])
        
        print '\rstm: ' + stm + '.',
        
# Add trolley baseline correction (with backwards correction)
print "\n\nAdding trolley baselines."

for st in np.arange(72):
    num_probes = len(trfp.STATION_PROBE_ID[st])
    for m in np.arange(num_probes):
        stm = 'st'+str(st)+',m'+str(m+1)
        
        def backwards_correction(time):
            c1 = tr_baseline_1[st, m]
            c2 = tr_baseline_2[st, m]
            t1 = baseline_time_1[st]
            t2 = baseline_time_2[st]
            return (c2-c1)/(t2-t1)*(time-t1) + c1
        
        correction = backwards_correction(vtr_df.index.values)
        vtr_df[stm] = vtr_df[stm] + correction
        correction = backwards_correction(vtr_tr_run_df.index.values)
        vtr_tr_run_df[stm] = vtr_tr_run_df[stm] + correction
        
        print '\rstm: ' + stm + '.',

print '\n\nDone.'

Appending fixed probe runs.
Appending run 3994.                              
Blinding fixed probe runs.

Appending trolley runs.
Appending run 3998.  
Blinding trolley runs.

Done appending runs.

Subtracting fixed probe baselines.
stm: st71,m4.   

Applying Jacobian.
stm: st71,m4.        

Adding trolley baselines.
stm: st71,m4.   

Done.


In [27]:
# bin into the agreed upon bins

bins = np.arange(1524383560, 1524640850, 10)-5
bin_centers = np.arange(1524383560, 1524640840, 10)

vtr_time_bin_df = vtr_df.groupby(pd.cut(vtr_df.index,bins)).mean()
vtr_time_bin_df.index = bin_centers

# print vtr_time_bin_df.iloc[[0,-1]]
# print vtr_time_bin_df.shape
# print bin_centers.shape

                  st0,m1      st0,m2     st0,m3     st0,m4     st0,m5  \
1524383560  51387.934494  102.257620 -99.983759 -34.886332  19.649595   
1524640830  51685.966226  117.162039 -88.144625 -30.440722   4.799612   

                st0,m6        st1,m1      st1,m2     st1,m3      st1,m4  \
1524383560 -840.135420  50538.854772  111.497681  61.863677  377.199355   
1524640830 -849.729965  50707.914760   73.247228  61.892301  390.237439   

             ...       st70,m3    st70,m4    st70,m5     st70,m6  \
1524383560   ...    -31.188585 -34.074374 -76.589752  549.439287   
1524640830   ...    -18.966847 -31.820435 -80.794836  550.602669   

                 st71,m1    st71,m2    st71,m3    st71,m4  st71,m5  st71,m6  
1524383560  51366.648972  60.701032 -42.875926  60.407979      NaN      NaN  
1524640830  51645.865840  71.785192 -34.941434  64.150779      NaN      NaN  

[2 rows x 432 columns]
(25728, 432)
(25728,)


In [28]:
test_df = vtr_time_bin_df.copy()

azi_avg_df = pd.DataFrame(np.zeros((test_df.shape[0],6)),
                         index = test_df.index,
                         columns = ['m' + str(m) for m in np.arange(6)+1])

weight = summed_azimuth_1+summed_azimuth_2
total_weight = np.sum(summed_azimuth_1 + summed_azimuth_2)

for m in np.arange(6):
    stm_list = ['st'+str(st)+',m'+str(m+1) for st in np.arange(72)]
    azi_avg_df['m'+str(m+1)] = test_df[stm_list].multiply(weight).sum(axis=1)/total_weight

In [43]:
review_df = azi_avg_df[['m1', 'm2', 'm3', 'm5']].copy()

review_df['m1_unc'] = 0.116
review_df['m2_unc'] = 0.060
review_df['m3_unc'] = 0.062
review_df['m5_unc'] = 0.100

review_df['m1'] = review_df['m1']/61.79
review_df['m2'] = review_df['m2']/61.79
review_df['m3'] = review_df['m3']/61.79
review_df['m5'] = review_df['m5']/61.79

review_df = review_df.reindex(sorted(review_df.columns), axis=1)

print review_df.head()

                    m1  m1_unc        m2  m2_unc        m3  m3_unc        m5  \
1524383560  840.945542   0.116  0.181946    0.06  0.075832   0.062 -0.827792   
1524383570  840.924363   0.116  0.182149    0.06  0.068415   0.062 -0.837025   
1524383580  840.908940   0.116  0.164082    0.06  0.065106   0.062 -0.845231   
1524383590  840.879869   0.116  0.137257    0.06  0.064849   0.062 -0.861169   
1524383600  841.015544   0.116  0.177587    0.06  0.066285   0.062 -0.816883   

            m5_unc  
1524383560     0.1  
1524383570     0.1  
1524383580     0.1  
1524383590     0.1  
1524383600     0.1  


In [45]:
review_df.to_csv('purcell_60hr.dat', sep=' ', index_label='time', float_format='%0.4f')